# Setup

###### commented out due to ODH notebook image error on direct pip commands. Use %horus error.
# run below commands in terminal
#!pip install --upgrade pip

In [1]:
#### import os
import pprint
import tempfile
import urllib
import tensorboard

import absl
import tensorflow as tf
import tensorflow_model_analysis as tfma
import tensorflow_decision_forests as tfdf
tf.get_logger().propagate = False
pp = pprint.PrettyPrinter()

%load_ext tfx.orchestration.experimental.interactive.notebook_extensions.skip

import tensorflow as tf
print('TensorFlow version: {}'.format(tf.__version__))
from tfx import v1 as tfx
print('TFX version: {}'.format(tfx.__version__))
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext

2022-08-19 18:44:10.785284: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-08-19 18:44:10.785335: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


TensorFlow version: 2.9.1
TFX version: 1.9.1


# Variables

In [2]:
import os

PIPELINE_NAME = "pipeline"
MODEL_NAME = "sepsis"

# Output directory to store artifacts generated from the pipeline.
PIPELINE_ROOT = os.path.join('../pipeline', PIPELINE_NAME)
# Path to a SQLite DB file to use as an MLMD storage.
METADATA_PATH = os.path.join('../pipeline/metadata', PIPELINE_NAME, 'metadata.db')
# Output directory where created models from the pipeline will be exported.
SERVING_MODEL_DIR = os.path.join('../models', MODEL_NAME)
# Path to the training data
DATA_TRAIN = '../data/training_data'
# File name
FILENAME='septic_data_labelled.csv'
RAW_DATA = os.path.join(DATA_TRAIN,FILENAME)

from absl import logging
logging.set_verbosity(logging.INFO)  # Set default logging level

print(PIPELINE_NAME)
print(PIPELINE_ROOT)
print(METADATA_PATH)
print(SERVING_MODEL_DIR)
print(DATA_TRAIN)
print(RAW_DATA)

pipeline
../pipeline/pipeline
../pipeline/metadata/pipeline/metadata.db
../models/sepsis
../data/training_data
../data/training_data/septic_data_labelled.csv


# View Data

In [3]:
!head {RAW_DATA}

Temp,HR,Resp,WBC,isSeptic
40.0,110.0,12.0,4.54,0
41.5,79.0,26.0,4.23,0
41.9,61.0,14.0,18.13,0
40.1,89.0,20.0,3.40,0
35.6,136.0,13.0,9.21,0
35.8,83.0,26.0,8.95,0
35.6,83.0,18.0,21.70,0
35.4,85.0,14.0,3.92,0
38.3,238.0,25.0,6.96,0


# Interactive Context

In [4]:
# Here, we create an InteractiveContext using default parameters. This will
# use a temporary directory with an ephemeral ML Metadata database instance.
# To use your own pipeline root or database, the optional properties
# `pipeline_root` and `metadata_connection_config` may be passed to
# InteractiveContext. Calls to InteractiveContext are no-ops outside of the
# notebook.
context = InteractiveContext(pipeline_root=PIPELINE_ROOT)

# ExampleGen

In [5]:
example_gen = tfx.components.CsvExampleGen(input_base=DATA_TRAIN)
context.run(example_gen, enable_cache=True)

INFO:absl:Running driver for CsvExampleGen
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:select span and version = (0, None)
INFO:absl:latest span and version = (0, None)
INFO:absl:Running executor for CsvExampleGen
INFO:absl:Generating examples.


INFO:absl:Processing input csv data ../data/training_data/* to TFExample.
INFO:absl:Examples generated.
INFO:absl:Running publisher for CsvExampleGen
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: CsvExampleGen
    execution_id: 1
    outputs:
        examples: OutputChannel(artifact_type=Examples, producer_component_id=CsvExampleGen, output_key=examples, additional_properties={}, additional_custom_properties={}))

In [6]:
artifact = example_gen.outputs['examples'].get()[0]
print(artifact.split_names, artifact.uri)

["train", "eval"] ../pipeline/pipeline/CsvExampleGen/examples/1


In [7]:
# Get the URI of the output artifact representing the training examples, which is a directory
train_uri = os.path.join(example_gen.outputs['examples'].get()[0].uri, 'Split-train')

# Get the list of files in this directory (all compressed TFRecord files)
tfrecord_filenames = [os.path.join(train_uri, name)
                      for name in os.listdir(train_uri)]

# Create a `TFRecordDataset` to read these files
dataset = tf.data.TFRecordDataset(tfrecord_filenames, compression_type="GZIP")

# Iterate over the first 3 records and decode them.
for tfrecord in dataset.take(1):
  serialized_example = tfrecord.numpy()
  example = tf.train.Example()
  example.ParseFromString(serialized_example)
  pp.pprint(example)

features {
  feature {
    key: "HR"
    value {
      float_list {
        value: 110.0
      }
    }
  }
  feature {
    key: "Resp"
    value {
      float_list {
        value: 12.0
      }
    }
  }
  feature {
    key: "Temp"
    value {
      float_list {
        value: 40.0
      }
    }
  }
  feature {
    key: "WBC"
    value {
      float_list {
        value: 4.539999961853027
      }
    }
  }
  feature {
    key: "isSeptic"
    value {
      int64_list {
        value: 0
      }
    }
  }
}



2022-08-19 18:44:50.064951: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-08-19 18:44:50.064997: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-08-19 18:44:50.065026: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (jupyterhub-nb-opentlc-2dmgr): /proc/driver/nvidia/version does not exist
2022-08-19 18:44:50.065302: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


# StatisticsGen

In [8]:
statistics_gen = tfx.components.StatisticsGen(
    examples=example_gen.outputs['examples'])
context.run(statistics_gen, enable_cache=True)

INFO:absl:Excluding no splits because exclude_splits is not set.
INFO:absl:Running driver for StatisticsGen
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Running executor for StatisticsGen
INFO:absl:Generating statistics for split train.
INFO:absl:Statistics for split train written to ../pipeline/pipeline/StatisticsGen/statistics/2/Split-train.
INFO:absl:Generating statistics for split eval.
INFO:absl:Statistics for split eval written to ../pipeline/pipeline/StatisticsGen/statistics/2/Split-eval.
INFO:absl:Running publisher for StatisticsGen
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: StatisticsGen
    execution_id: 2
    outputs:
        statistics: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=StatisticsGen, output_key=statistics, additional_properties={}, additional_custom_properties={}))

In [9]:
context.show(statistics_gen.outputs['statistics'])

# SchemaGen

In [10]:
schema_gen = tfx.components.SchemaGen(
    statistics=statistics_gen.outputs['statistics'],
    infer_feature_shape=False)
context.run(schema_gen, enable_cache=True)

INFO:absl:Excluding no splits because exclude_splits is not set.
INFO:absl:Running driver for SchemaGen
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Running executor for SchemaGen
INFO:absl:Processing schema from statistics for split train.
INFO:absl:Processing schema from statistics for split eval.
INFO:absl:Schema written to ../pipeline/pipeline/SchemaGen/schema/3/schema.pbtxt.
INFO:absl:Running publisher for SchemaGen
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: SchemaGen
    execution_id: 3
    outputs:
        schema: OutputChannel(artifact_type=Schema, producer_component_id=SchemaGen, output_key=schema, additional_properties={}, additional_custom_properties={}))

In [11]:
context.show(schema_gen.outputs['schema'])

,Type,Presence,Valency,Domain
Feature name,,,,
'HR',FLOAT,required,single,-
'Resp',FLOAT,required,single,-
'Temp',FLOAT,required,single,-
'WBC',FLOAT,required,,-
'isSeptic',INT,required,single,-


# ExampleValidator

In [12]:
example_validator = tfx.components.ExampleValidator(
    statistics=statistics_gen.outputs['statistics'],
    schema=schema_gen.outputs['schema'])
context.run(example_validator, enable_cache=True)

INFO:absl:Excluding no splits because exclude_splits is not set.
INFO:absl:Running driver for ExampleValidator
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Running executor for ExampleValidator
INFO:absl:Validating schema against the computed statistics for split train.
INFO:absl:Validation complete for split train. Anomalies written to ../pipeline/pipeline/ExampleValidator/anomalies/4/Split-train.
INFO:absl:Validating schema against the computed statistics for split eval.
INFO:absl:Validation complete for split eval. Anomalies written to ../pipeline/pipeline/ExampleValidator/anomalies/4/Split-eval.
INFO:absl:Running publisher for ExampleValidator
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: ExampleValidator
    execution_id: 4
    outputs:
        anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=ExampleValidator, output_key=anomalies, additional_properties={}, additional_custom_properties={}))

In [13]:
context.show(example_validator.outputs['anomalies'])

# Transform

In [14]:
_vitals_constants_module_file = '../src/vitals_constants.py'

In [15]:
%%writefile {_vitals_constants_module_file}

NUMERICAL_FEATURES = ['Temp', 'HR', 'Resp', 'WBC']

# Keys
LABEL_KEY = 'isSeptic'

def t_name(key):
  """
  Rename the feature keys so that they don't clash with the raw keys when
  running the Evaluator component.
  Args:
    key: The original feature key
  Returns:
    key with '_xf' appended
  """
  return key + '_xf'

Writing ../src/vitals_constants.py


## Pre-process

In [16]:
_vitals_transform_module_file = '../src/vitals_transform.py'

In [17]:
%%writefile {_vitals_transform_module_file}

import tensorflow as tf
import tensorflow_transform as tft

# Imported files such as vitals_constants are normally cached, so changes are
# not honored after the first import.  Normally this is good for efficiency, but
# during development when we may be iterating code it can be a problem. To
# avoid this problem during development, reload the file.
import vitals_constants
import sys
import importlib
importlib.reload(vitals_constants)

_NUMERICAL_FEATURES = vitals_constants.NUMERICAL_FEATURES
_LABEL_KEY = vitals_constants.LABEL_KEY


def _fill_in_missing(x):
  """Replace missing values in a SparseTensor.
  Fills in missing values of `x` with '' or 0, and converts to a dense tensor.
  Args:
    x: A `SparseTensor` of rank 2.  Its dense shape should have size at most 1
      in the second dimension.
  Returns:
    A rank 1 tensor where missing values of `x` have been filled in.
  """
  if not isinstance(x, tf.sparse.SparseTensor):
    return x

  default_value = '' if x.dtype == tf.string else 0
  return tf.squeeze(
      tf.sparse.to_dense(
          tf.SparseTensor(x.indices, x.values, [x.dense_shape[0], 1]),
          default_value),
      axis=1)


def preprocessing_fn(inputs):
  """tf.transform's callback function for preprocessing inputs.
  Args:
    inputs: map from feature keys to raw not-yet-transformed features.
  Returns:
    Map from string feature key to transformed feature operations.
  """
  outputs = {}
  for key in _NUMERICAL_FEATURES:
    # If sparse make it dense, setting nan's to 0 or '', and apply zscore.
    outputs[vitals_constants.t_name(key)] = tft.scale_to_z_score(
        _fill_in_missing(inputs[key]), name=key)

  outputs[_LABEL_KEY] = _fill_in_missing(inputs[_LABEL_KEY]) 
    
  return outputs

Writing ../src/vitals_transform.py


## Transform Data

In [18]:
transform = tfx.components.Transform(
    examples=example_gen.outputs['examples'],
    schema=schema_gen.outputs['schema'],
    module_file=os.path.abspath(_vitals_transform_module_file))
context.run(transform, enable_cache=True)

INFO:absl:Generating ephemeral wheel package for '/opt/app-root/src/mlops-basic/src/vitals_transform.py' (including modules: ['vitals_transform', 'vitals_constants']).
INFO:absl:User module package has hash fingerprint version 16b8e681eefa7c946a150147466cc447ddfce8a7c23b0eeba83da96ceaca9845.
INFO:absl:Executing: ['/opt/app-root/bin/python3.8', '/tmp/tmpo6eknfom/_tfx_generated_setup.py', 'bdist_wheel', '--bdist-dir', '/tmp/tmp867zsgn6', '--dist-dir', '/tmp/tmp_xtkztnb']


running bdist_wheel
running build
running build_py
creating build
creating build/lib
copying vitals_transform.py -> build/lib
copying vitals_constants.py -> build/lib


/opt/app-root/lib64/python3.8/site-packages/setuptools/command/install.py:34: SetuptoolsDeprecationWarning: setup.py install is deprecated. Use build and pip and other standards-based tools.
  warnings.warn(


installing to /tmp/tmp867zsgn6
running install
running install_lib
copying build/lib/vitals_transform.py -> /tmp/tmp867zsgn6
copying build/lib/vitals_constants.py -> /tmp/tmp867zsgn6
running install_egg_info
running egg_info
creating tfx_user_code_Transform.egg-info
writing tfx_user_code_Transform.egg-info/PKG-INFO
writing dependency_links to tfx_user_code_Transform.egg-info/dependency_links.txt


INFO:absl:Successfully built user code wheel distribution at '../pipeline/pipeline/_wheels/tfx_user_code_Transform-0.0+16b8e681eefa7c946a150147466cc447ddfce8a7c23b0eeba83da96ceaca9845-py3-none-any.whl'; target user module is 'vitals_transform'.
INFO:absl:Full user module path is 'vitals_transform@../pipeline/pipeline/_wheels/tfx_user_code_Transform-0.0+16b8e681eefa7c946a150147466cc447ddfce8a7c23b0eeba83da96ceaca9845-py3-none-any.whl'
INFO:absl:Running driver for Transform
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Running executor for Transform
INFO:absl:Analyze the 'train' split and transform all splits when splits_config is not set.
INFO:absl:udf_utils.get_fn {'module_file': None, 'module_path': 'vitals_transform@../pipeline/pipeline/_wheels/tfx_user_code_Transform-0.0+16b8e681eefa7c946a150147466cc447ddfce8a7c23b0eeba83da96ceaca9845-py3-none-any.whl', 'preprocessing_fn': None} 'preprocessing_fn'
INFO:absl:Installing '../pipeline/pipeline/_wheels/tfx_user_code_Tr

...

Instructions for updating:
Use ref() instead.


INFO:absl:Feature HR has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature Resp has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature Temp has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature WBC has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature isSeptic has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature HR has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature Resp has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature Temp has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature WBC has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature isSeptic has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature HR has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature Resp has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature Temp has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature WBC has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature isSeptic has no shape. Setting 

INFO:tensorflow:Assets written to: ../pipeline/pipeline/Transform/transform_graph/5/.temp_path/tftransform_tmp/be44c1d5266942d498d3b578f98272b8/assets
INFO:tensorflow:struct2tensor is not available.
INFO:tensorflow:tensorflow_text is not available.
INFO:tensorflow:Assets written to: ../pipeline/pipeline/Transform/transform_graph/5/.temp_path/tftransform_tmp/d8dd310857904e76baf024d9b631a14a/assets
INFO:tensorflow:struct2tensor is not available.
INFO:tensorflow:tensorflow_text is not available.
INFO:tensorflow:struct2tensor is not available.
INFO:tensorflow:tensorflow_text is not available.


INFO:absl:Running publisher for Transform
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: Transform
    execution_id: 5
    outputs:
        transform_graph: OutputChannel(artifact_type=TransformGraph, producer_component_id=Transform, output_key=transform_graph, additional_properties={}, additional_custom_properties={})
        transformed_examples: OutputChannel(artifact_type=Examples, producer_component_id=Transform, output_key=transformed_examples, additional_properties={}, additional_custom_properties={})
        updated_analyzer_cache: OutputChannel(artifact_type=TransformCache, producer_component_id=Transform, output_key=updated_analyzer_cache, additional_properties={}, additional_custom_properties={})
        pre_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=pre_transform_schema, additional_properties={}, additional_custom_properties={})
        pre_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=pre_transform_stats, additional_properties={}, additional_custom_properties={})
        post_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=post_transform_schema, additional_properties={}, additional_custom_properties={})
        post_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=post_transform_stats, additional_properties={}, additional_custom_properties={})
        post_transform_anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=Transform, output_key=post_transform_anomalies, additional_properties={}, additional_custom_properties={}))

In [19]:
# Get the URI of the output artifact representing the transformed examples, which is a directory
train_uri = os.path.join(transform.outputs['transformed_examples'].get()[0].uri, 'Split-train')

# Get the list of files in this directory (all compressed TFRecord files)
tfrecord_filenames = [os.path.join(train_uri, name)
                      for name in os.listdir(train_uri)]

# Create a `TFRecordDataset` to read these files
dataset = tf.data.TFRecordDataset(tfrecord_filenames, compression_type="GZIP")

# Iterate over the first 1 records and decode them.
for tfrecord in dataset.take(1):
  serialized_example = tfrecord.numpy()
  example = tf.train.Example()
  example.ParseFromString(serialized_example)
  pp.pprint(example)

features {
  feature {
    key: "HR_xf"
    value {
      float_list {
        value: -0.12786640226840973
      }
    }
  }
  feature {
    key: "Resp_xf"
    value {
      float_list {
        value: -1.2934118509292603
      }
    }
  }
  feature {
    key: "Temp_xf"
    value {
      float_list {
        value: 0.8965023756027222
      }
    }
  }
  feature {
    key: "WBC_xf"
    value {
      float_list {
        value: -0.8832799196243286
      }
    }
  }
  feature {
    key: "isSeptic"
    value {
      int64_list {
        value: 0
      }
    }
  }
}



# Trainer

In [20]:
_vitals_trainer_module_file = '../src/vitals_trainer.py'

In [21]:
%%writefile {_vitals_trainer_module_file}

from typing import Dict, List, Text

import os
import glob
from absl import logging

import datetime
import tensorflow as tf
import tensorflow_transform as tft
import tensorflow_decision_forests as tfdf


from tfx import v1 as tfx
from tfx_bsl.public import tfxio
from tensorflow_transform import TFTransformOutput

# Imported files such as vitals_constants are normally cached, so changes are
# not honored after the first import.  Normally this is good for efficiency, but
# during development when we may be iterating code it can be a problem. To
# avoid this problem during development, reload the file.
import vitals_constants
import sys
import importlib
importlib.reload(vitals_constants)

_LABEL_KEY = vitals_constants.LABEL_KEY

_BATCH_SIZE = 40


def _input_fn(file_pattern: List[Text],
              data_accessor: tfx.components.DataAccessor,
              tf_transform_output: tft.TFTransformOutput,
              batch_size: int = 200) -> tf.data.Dataset:
  """Generates features and label for tuning/training.

  Args:
    file_pattern: List of paths or patterns of input tfrecord files.
    data_accessor: DataAccessor for converting input to RecordBatch.
    tf_transform_output: A TFTransformOutput.
    batch_size: representing the number of consecutive elements of returned
      dataset to combine in a single batch

  Returns:
    A dataset that contains (features, indices) tuple where features is a
      dictionary of Tensors, and indices is a single Tensor of label indices.
  """
  return data_accessor.tf_dataset_factory(
      file_pattern,
      tfxio.TensorFlowDatasetOptions(
          batch_size=batch_size, label_key=_LABEL_KEY),
      tf_transform_output.transformed_metadata.schema)

def _get_tf_examples_serving_signature(model, tf_transform_output):
  """Returns a serving signature that accepts `tensorflow.Example`."""

  # We need to track the layers in the model in order to save it.
  # TODO(b/162357359): Revise once the bug is resolved.
  model.tft_layer_inference = tf_transform_output.transform_features_layer()

  @tf.function(input_signature=[
      tf.TensorSpec(shape=[None], dtype=tf.string, name='examples')
  ])
  def serve_tf_examples_fn(serialized_tf_example):
    """Returns the output to be used in the serving signature."""
    raw_feature_spec = tf_transform_output.raw_feature_spec()
    # Remove label feature since these will not be present at serving time.
    raw_feature_spec.pop(_LABEL_KEY)
    raw_features = tf.io.parse_example(serialized_tf_example, raw_feature_spec)
    transformed_features = model.tft_layer_inference(raw_features)
    logging.info('serve_transformed_features = %s', transformed_features)

    outputs = model(transformed_features)
    # TODO(b/154085620): Convert the predicted labels from the model using a
    # reverse-lookup (opposite of transform.py).
    return {'outputs': outputs}

  return serve_tf_examples_fn


def _get_transform_features_signature(model, tf_transform_output):
  """Returns a serving signature that applies tf.Transform to features."""

  # We need to track the layers in the model in order to save it.
  # TODO(b/162357359): Revise once the bug is resolved.
  model.tft_layer_eval = tf_transform_output.transform_features_layer()

  @tf.function(input_signature=[
      tf.TensorSpec(shape=[None], dtype=tf.string, name='examples')
  ])
  def transform_features_fn(serialized_tf_example):
    """Returns the transformed_features to be fed as input to evaluator."""
    raw_feature_spec = tf_transform_output.raw_feature_spec()
    raw_features = tf.io.parse_example(serialized_tf_example, raw_feature_spec)
    transformed_features = model.tft_layer_eval(raw_features)
    logging.info('eval_transformed_features = %s', transformed_features)
    return transformed_features

  return transform_features_fn


def export_serving_model(tf_transform_output, model, output_dir):
  """Exports a keras model for serving.
  Args:
    tf_transform_output: Wrapper around output of tf.Transform.
    model: A keras model to export for serving.
    output_dir: A directory where the model will be exported to.
  """
  # The layer has to be saved to the model for keras tracking purpases.
  model.tft_layer = tf_transform_output.transform_features_layer()

  signatures = {
      'serving_default':
          _get_tf_examples_serving_signature(model, tf_transform_output),
      'transform_features':
          _get_transform_features_signature(model, tf_transform_output),
  }

  model.save(output_dir, save_format='tf', signatures=signatures)

def run_fn(fn_args: tfx.components.FnArgs):
  """Train the model based on given args.

  Args:
    fn_args: Holds args used to train the model as name/value pairs.
  """

  tf_transform_output = tft.TFTransformOutput(fn_args.transform_output)

  train_dataset = _input_fn(fn_args.train_files, fn_args.data_accessor, 
                            tf_transform_output, _BATCH_SIZE)
  eval_dataset = _input_fn(fn_args.eval_files, fn_args.data_accessor, 
                           tf_transform_output, _BATCH_SIZE)
    
  model = tfdf.keras.RandomForestModel()
  model.fit(tf_transform_output)
    
  # Export the model.
  export_serving_model(tf_transform_output, model, fn_args.serving_model_dir)

Writing ../src/vitals_trainer.py


In [22]:
trainer = tfx.components.Trainer(
    module_file=os.path.abspath(_vitals_trainer_module_file),
    examples=transform.outputs['transformed_examples'],
    transform_graph=transform.outputs['transform_graph'],
    schema=schema_gen.outputs['schema'],
    train_args=tfx.proto.TrainArgs(num_steps=5000),
    eval_args=tfx.proto.EvalArgs(num_steps=1000))
context.run(trainer, enable_cache=True)

INFO:absl:Generating ephemeral wheel package for '/opt/app-root/src/mlops-basic/src/vitals_trainer.py' (including modules: ['vitals_transform', 'vitals_constants', 'vitals_trainer']).
INFO:absl:User module package has hash fingerprint version 976162edc0755e8ce96d22eb9db3968032ac7563a27da5de68c0c033b0f73d6d.
INFO:absl:Executing: ['/opt/app-root/bin/python3.8', '/tmp/tmp9peh4y5h/_tfx_generated_setup.py', 'bdist_wheel', '--bdist-dir', '/tmp/tmp855ed_fk', '--dist-dir', '/tmp/tmpmssj616_']


running bdist_wheel
running build
running build_py
creating build
creating build/lib
copying vitals_transform.py -> build/lib
copying vitals_constants.py -> build/lib
copying vitals_trainer.py -> build/lib


/opt/app-root/lib64/python3.8/site-packages/setuptools/command/install.py:34: SetuptoolsDeprecationWarning: setup.py install is deprecated. Use build and pip and other standards-based tools.
  warnings.warn(


installing to /tmp/tmp855ed_fk
running install
running install_lib
copying build/lib/vitals_transform.py -> /tmp/tmp855ed_fk
copying build/lib/vitals_constants.py -> /tmp/tmp855ed_fk
copying build/lib/vitals_trainer.py -> /tmp/tmp855ed_fk
running install_egg_info
running egg_info
creating tfx_user_code_Trainer.egg-info
writing tfx_user_code_Trainer.egg-info/PKG-INFO
writing dependency_links to tfx_user_code_Trainer.egg-info/dependency_links.txt


INFO:absl:Successfully built user code wheel distribution at '../pipeline/pipeline/_wheels/tfx_user_code_Trainer-0.0+976162edc0755e8ce96d22eb9db3968032ac7563a27da5de68c0c033b0f73d6d-py3-none-any.whl'; target user module is 'vitals_trainer'.
INFO:absl:Full user module path is 'vitals_trainer@../pipeline/pipeline/_wheels/tfx_user_code_Trainer-0.0+976162edc0755e8ce96d22eb9db3968032ac7563a27da5de68c0c033b0f73d6d-py3-none-any.whl'
INFO:absl:Running driver for Trainer
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Running executor for Trainer
INFO:absl:Train on the 'train' split when train_args.splits is not set.
INFO:absl:Evaluate on the 'eval' split when eval_args.splits is not set.
INFO:absl:udf_utils.get_fn {'train_args': '{\n  "num_steps": 5000\n}', 'eval_args': '{\n  "num_steps": 1000\n}', 'module_file': None, 'run_fn': None, 'trainer_fn': None, 'custom_config': 'null', 'module_path': 'vitals_trainer@../pipeline/pipeline/_wheels/tfx_user_code_Trainer-0.0+976162edc0755

...

Use /tmp/tmpz0vt7b7d as temporary training directory


INFO:absl:Use /tmp/tmpz0vt7b7d as temporary training directory


Reading training dataset...


INFO:absl:Reading training dataset...


ValueError: Failed to find data adapter that can handle input: <class 'tensorflow_transform.output_wrapper.TFTransformOutput'>, <class 'NoneType'>

# Evaluator

In [ ]:
# Imported files such as vitals_constants are normally cached, so changes are
# not honored after the first import.  Normally this is good for efficiency, but
# during development when we may be iterating code it can be a problem. To
# avoid this problem during development, reload the file.
import sys
sys.path.append('../src/')
import vitals_constants
import importlib
importlib.reload(vitals_constants)

eval_config = tfma.EvalConfig(
    model_specs=[
        # This assumes a serving model with signature 'serving_default'. If
        # using estimator based EvalSavedModel, add signature_name: 'eval' and
        # remove the label_key.
        tfma.ModelSpec(
            signature_name='serving_default',
            label_key=vitals_constants.LABEL_KEY,
            preprocessing_function_names=['transform_features'],
            )
        ],
    metrics_specs=[
        tfma.MetricsSpec(
            # The metrics added here are in addition to those saved with the
            # model (assuming either a keras model or EvalSavedModel is used).
            # Any metrics added into the saved model (for example using
            # model.compile(..., metrics=[...]), etc) will be computed
            # automatically.
            # To add validation thresholds for metrics saved with the model,
            # add them keyed by metric name to the thresholds map.
            metrics=[
                tfma.MetricConfig(class_name='ExampleCount'),
                tfma.MetricConfig(class_name='BinaryAccuracy',
                  threshold=tfma.MetricThreshold(
                      value_threshold=tfma.GenericValueThreshold(
                          lower_bound={'value': 0.5}),
                      # Change threshold will be ignored if there is no
                      # baseline model resolved from MLMD (first run).
                      change_threshold=tfma.GenericChangeThreshold(
                          direction=tfma.MetricDirection.HIGHER_IS_BETTER,
                          absolute={'value': -1e-10})))
            ]
        )
    ],
    slicing_specs=[
        # An empty slice spec means the overall slice, i.e. the whole dataset.
        tfma.SlicingSpec(),
        # Data can be sliced along a feature column. In this case, data is
        # sliced along feature column HR.
        tfma.SlicingSpec(
            feature_keys=['HR'])
    ])

In [ ]:
# Use TFMA to compute a evaluation statistics over features of a model and
# validate them against a baseline.

# The model resolver is only required if performing model validation in addition
# to evaluation. In this case we validate against the latest blessed model. If
# no model has been blessed before (as in this case) the evaluator will make our
# candidate the first blessed model.
model_resolver = tfx.dsl.Resolver(
      strategy_class=tfx.dsl.experimental.LatestBlessedModelStrategy,
      model=tfx.dsl.Channel(type=tfx.types.standard_artifacts.Model),
      model_blessing=tfx.dsl.Channel(
          type=tfx.types.standard_artifacts.ModelBlessing)).with_id(
              'latest_blessed_model_resolver')
context.run(model_resolver, enable_cache=True)

evaluator = tfx.components.Evaluator(
    examples=example_gen.outputs['examples'],
    model=trainer.outputs['model'],
    baseline_model=model_resolver.outputs['model'],
    eval_config=eval_config)
context.run(evaluator, enable_cache=True)

In [ ]:
import tensorflow_model_analysis as tfma

# Get the TFMA output result path and load the result.
PATH_TO_RESULT = evaluator.outputs['evaluation'].get()[0].uri
tfma_result = tfma.load_eval_result(PATH_TO_RESULT)

# Show data sliced along feature column HR.
tfma.view.render_slicing_metrics(
    tfma_result, slicing_column='HR')

In [ ]:
blessing_uri = evaluator.outputs['blessing'].get()[0].uri
!ls -l {blessing_uri}

# Pusher

In [ ]:
pusher = tfx.components.Pusher(
    model=trainer.outputs['model'],
    model_blessing=evaluator.outputs['blessing'],
    push_destination=tfx.proto.PushDestination(
        filesystem=tfx.proto.PushDestination.Filesystem(
            base_directory=SERVING_MODEL_DIR)))
context.run(pusher, enable_cache=True)

In [ ]:
push_uri = pusher.outputs['pushed_model'].get()[0].uri
model = tf.saved_model.load(push_uri)

for item in model.signatures.items():
  pp.pprint(item)

# BulkInferrer

In [ ]:
INFERENCE_ROOT='../data/serving_data'
INFERENCE_EXAMPLES = os.path.join(INFERENCE_ROOT, 'septic_data_unlabelled.csv')
!head {INFERENCE_EXAMPLES}

In [ ]:
from tfx.proto import example_gen_pb2

output = tfx.proto.Output(
             split_config=example_gen_pb2.SplitConfig(splits=[
                 tfx.proto.SplitConfig.Split(name='unlabelled', hash_buckets=1)             ]))

inference_example_gen = tfx.components.CsvExampleGen(
    # input_base an external directory containing the CSV files.
    input_base=INFERENCE_ROOT, 
    # To customize the train/eval split ratio which ExampleGen will output, set the output_config for ExampleGen component. 
    output_config=output)
context.run(inference_example_gen, enable_cache=True)

In [ ]:
# https://github.com/tensorflow/tfx/issues/2478#issuecomment-770373362

bulk_inferrer = tfx.components.BulkInferrer(
    examples=inference_example_gen.outputs['examples'],
    model=trainer.outputs['model'],
    model_blessing=evaluator.outputs['blessing'],
    data_spec=tfx.proto.DataSpec(example_splits=['unlabelled']),
    model_spec=tfx.proto.ModelSpec())

context.run(bulk_inferrer)

In [ ]:
# Get the URI of the output artifact representing the transformed examples, which is a directory
# train_uri = os.path.join(transform.outputs['transformed_examples'].get()[0].uri, 'Split-train')
inference_uri = bulk_inferrer.outputs['inference_result'].get()[0].uri

# Get the list of files in this directory (all compressed TFRecord files)
tfrecord_filenames = [os.path.join(inference_uri, name)
                      for name in os.listdir(inference_uri)]

# Create a `TFRecordDataset` to read these files
dataset = tf.data.TFRecordDataset(tfrecord_filenames, compression_type="GZIP")

# Iterate over the first 1 records and decode them.
for tfrecord in dataset.take(2):
  serialized_example = tfrecord.numpy()
  inference_example = tf.train.Example()
  inference_example.ParseFromString(serialized_example)
  pp.pprint(example)